Load the packages required to define `JuMP` model

In [1]:
using JuMP
import Gurobi; const GRB_ENV = Gurobi.Env();

Academic license - for non-commercial use only


The function creates a model with 3 objectives

In [2]:
function create_eps_model()
    Annual_Demand = 64000.0
    df = [0.6, 0.3, 0.1]
    demand = df .* 64000.0
    pData = [31000 15000 22000 10000; 30 75 60 90; 1.44 0.72 0.45 0]
    PLset = collect(Iterators.product(1:4, 1:3))[[2,8,9]]
    m = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), 
            "NonConvex" => 1, "OutputFlag" => 0))
    @variables(m, begin
            x[1:4,1:3] ≥ 0
            Z[1:3]
    end)

    @constraints(m, begin
                objcost, sum(pData[2,:]'*x) == Z[1]
                objCO2, sum(pData[3,:]'*x) == Z[2]
                objEndo, sum(x[[1,4],:]) == -Z[3]
                xzero_set[p=1:4,l=1:3; (p,l) in PLset], x[p,l] == 0
                defcap, sum(x; dims=2) .≤ pData[1, :]
                defdem, sum(x; dims=1)[:] .≥ demand
    end)
    return m, Z
end

create_eps_model (generic function with 1 method)

In [3]:
m, Z = create_eps_model();

Calling on `MOJuMP` to use `eff_pareto` function and finding pareto optimal points.

> Notice how some warning are being thrown. This means that corresponding points may not be pareto optimal and needs to be inspected further

In [4]:
import Pkg; Pkg.activate("../.")
using MOJuMP

pareto, _ = eff_pareto(m, Z);

Activating environment at `~/MOJuMP/Project.toml`


┌ Warning: Slack not zero for eps=[47340.0, -32250.0]
└ @ MOJuMP /home/ushah/MOJuMP/src/MOJuMP.jl:107
┌ Warning: Slack not zero for eps=[48420.0, -36625.0]
└ @ MOJuMP /home/ushah/MOJuMP/src/MOJuMP.jl:107
┌ Warning: Slack not zero for eps=[50579.99999999999, -38375.0]
└ @ MOJuMP /home/ushah/MOJuMP/src/MOJuMP.jl:107
┌ Warning: Slack not zero for eps=[51659.99999999999, 3.3675e6]
└ @ MOJuMP /home/ushah/MOJuMP/src/MOJuMP.jl:126
┌ Warning: Slack not zero for eps=[55979.99999999999, 3.1725e6]
└ @ MOJuMP /home/ushah/MOJuMP/src/MOJuMP.jl:126


This part is just to print the results in a pretty manner

In [5]:
using DataFrames
pareto |> DataFrame

,x1,x2,x3
,Float64,Float64,Float64
1,3.315e6,53820.0,-39000.0
2,3.1725e6,57780.0,-37500.0
3,3.585e6,49500.0,-33000.0
4,3.45e6,51660.0,-36000.0
5,3.12e6,60300.0,-34000.0
6,3.21e6,55980.0,-40000.0
7,3.72e6,47340.0,-30000.0
8,3.6525e6,48420.0,-31500.0
9,3.1875e6,57060.0,-38500.0
